# Laboratorio de BackPropagation

En el presente laboratorio Ud. debe diseñar una red neuronal con dos capas intermedias para el problema de reconocimiento de dígitos manuscritos. Para ello debe codear las funciones de:
1. Forward propagation (04 puntos)
2. Función de costo (02 puntos)
3. Back-propagation (08 puntos)
4. Predicción (02 puntos)
5. Qué parámetros hacen que Ud. encuentre el mejor resultado? (04 puntos)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('ex3data1.mat')

In [2]:
X = data['X']
y = data['y']

X.shape, y.shape

((5000, 400), (5000, 1))

In [3]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [4]:
y[0], y_onehot[0,:]

(array([10], dtype=uint8),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]))

In [5]:
# initial setup
input_size = 400
hidden_size1 = 20
hidden_size2 = 30
num_labels = 10
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1) + num_labels * (hidden_size2 + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1):hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1)], (hidden_size2, (hidden_size1 + 1))))
theta3 = np.matrix(np.reshape(params[hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1):], (num_labels, (hidden_size2 + 1))))

theta1.shape, theta2.shape, theta3.shape

((20, 401), (30, 21), (10, 31))

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [9]:
def forward_propagate(X, theta1, theta2, theta3):
    #M=5000
    m = X.shape[0]
    #Insert a 1 at the start of the initial variables (biased number)
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    #Generate Activation values that takes us from layer 1 to layer 2, save them in z2
    z2 = a1 * theta1.T
    #Apply sigmoid function to every activation value (Z2), and
    #Insert a 1 at the start of these activation values (biased number)
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    #Generate Activation value that takes us from layer 2 to layer 3, save them in z3
    z3 = a2 * theta2.T
    #Apply sigmoid function to every activation value (Z3), and
    #Insert a 1 at the start of these activation values (biased number)
    a3 = np.insert(sigmoid(z3), 0, values=np.ones(m), axis=1)
    #Generate Activation value that takes us from layer 3 to layer 4 (final one), and save them in z4
    z4 = a3 * theta3.T
    #Apply Sigmoid Function to z4, and that's our answer
    h = sigmoid(z4)
    return a1, z2, a2, z3, a3, z4, h

In [10]:
#A1 = Layer 1 (initial) but with the biased number at the start
#Z2 = Activation values at layer 2
#A2 = Activation values at layer 2 but with a sigmoid applied to them, plus a biased number at the start
#Z3 = Activation values at layer 3
#A3 = Activation values at layer 3 but with a sigmoid applied to them, plus a biased number at the start
#Z4 = Activation value at layer 4 (final one)
#H = Final result
a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
a1.shape, z2.shape, a2.shape, z3.shape, a3.shape, z4.shape, h.shape

((5000, 401),
 (5000, 20),
 (5000, 21),
 (5000, 30),
 (5000, 31),
 (5000, 10),
 (5000, 10))

In [19]:
def cost(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    #M = 5000
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # unravel the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1):hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1)], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1):], (num_labels, (hidden_size2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
    
    # compute the cost
    J = 0
    # Using the formula, we iterate through every example
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))
    
    return J

In [20]:
cost(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)

6.7926084664272075

In [21]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [71]:
def backprop(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # unravel the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size1 * (input_size + 1):hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1)], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1):], (num_labels, (hidden_size2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)
    
    # Using the formula, we iterate through every example
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))
    
    
    #With cost already calculated,
    #perform backpropagation
    for t in range(m):
        #For every example
        #A1T = Data of example t (layer 1)
        a1t = a1[t,:]
        #Z2T = Activation values at layer 2 of example t
        z2t = z2[t,:]  
        #A2T = Sigmoided Z2T of example t
        a2t = a2[t,:]  
        #Z3T = Activation values at layer 3 of example t
        z3t = z3[t,:]
        #A3T = Activation values at layer 3 but with a sigmoid applied to them, plus a biased number at the start
        a3t = a3[t,:]
        #Z4T = Activation value at layer 4 (final one)
        z4t = z4[t,:]
        #HT = Final Result of example t
        ht = h[t,:]  # (1, 10)
        #Y = Actual Result of example t
        yt = y[t,:]  # (1, 10)
        #D4T = Error at example t
        d4t = ht - yt  # (1, 10)
        
        #Insert Ones in Z3T
        z3t = np.insert(z3t, 0, values=np.ones(1))
        #Multiply elements of D4T by theta3 and by the sigmoided Z3T, you get D3T
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(z3t))
        
        #Insert Ones in Z2T
        z2t = np.insert(z2t, 0, values=np.ones(1))
        #Multiply elements of D3T by theta2 and by the sigmoided Z2T, you get D2T
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(z2t))
        #That 
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + (d3t[:,1:]).T * a2t
        delta3 = delta3 + d4t.T * a3t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    delta3[:,1:] = delta3[:,1:] + (theta3[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))
    
    return J, grad

In [72]:
J, grad = backprop(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(6.7926084664272075, (8960,))

In [101]:
from scipy.optimize import minimize
learning_rate = 0.5
# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size1,hidden_size2, num_labels, X, y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 0.19188104219025026
     jac: array([ -5.05260256e-04,  -2.73150268e-06,   9.01410762e-07, ...,
         1.35620341e-04,   1.32223448e-04,   2.29977941e-04])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 25
  status: 3
 success: False
       x: array([ 0.79726096, -0.02731503,  0.00901411, ..., -0.51868121,
        0.3354375 ,  0.19514874])

In [102]:
X = np.matrix(X)
# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size1 * (input_size + 1):hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1)], (hidden_size2, (hidden_size1 + 1))))
theta3 = np.matrix(np.reshape(fmin.x[hidden_size1*(input_size+1)+hidden_size2*(hidden_size1+1):], (num_labels, (hidden_size2 + 1))))

a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
y_pred = np.array(np.argmax(h, axis=1) + 1)
print h[125]

[[  1.29944840e-07   6.94214124e-03   7.68360181e-07   4.70169943e-10
    4.17845484e-05   7.72595901e-04   3.33022216e-05   5.01719598e-04
    6.78750055e-05   9.98905833e-01]]


In [103]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)

accuracy = 99.84%


In [ ]:
###############################################################
#      Que parametros generan los mejores resultados?         #
###############################################################
#  Learning Rate      Capa1       Capa2      Accuracy
#     1               20          30           99.1
#     0.8             20          30           99.72
#     0.5             20          30           99.84

#Investigacion:
#Dada una tasa de aprendizaje alta, como lo es 0.5, utilizar varios elementos en cada capa resulta ser muy ineficiente.[1]
#De esta manera, concluimos que el numero de unidades escondidas (50), resulta más efectivo que números elevados como 600,
#aún sin haberlo probado en nuestro modelo (lo cual tardaría bastante tiempo, además).

#[1] David Stutz, Pavel Golik, Ralf Schlüter, and Hermann Ney. Introduction to Neural Networks. 
#Seminar on Selected Topics in Human Language Technology and Pattern Recognition, 2014.
#Encontrado en: http://davidstutz.de/seminar-paper-introduction-neural-networks/

